## Introduction to Natural Language Processing
[**CC-BY-NC-SA**](https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en)<br/>
Prof. Dr. Annemarie Friedrich<br/>
Faculty of Applied Computer Science, University of Augsburg<br/>
Date: **SS 2025**

# 11. Sequence Labeling (Homework)

__Recommendation:__ Use a GPU for notebook, e.g., in Google Colab Runtime --> Change Runtime --> GPU --> T4.

**Learning Goals**

* Explain part-of-speech tagging
* Explain named entity recognition
* Implement masking in PyTorch
* Train and evaluate a sequence labeling model

❗ Upon completion, upload your code (this notebook) to your GitLab repository.

In [ ]:
# Installations
!pip install -U datasets
!pip install transformers
!pip install seqeval
!pip install evaluate

# Imports
import numpy as np
import random
import os

import torch
import transformers
import evaluate
from torch import optim
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

# Define the device we'll use for tensor computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Computing on:", device)

# Should we still have some source for non-determinism in our code, this will complain:
torch.use_deterministic_algorithms(True, warn_only=True)

## Named Entity Recognition

In this homework, we will work with the CoNLL 2003 Named Entity dataset. The dataset was developed for a shared task described in [Introduction to the CoNLL-2003 Shared Task: Language-Independent Named Entity Recognition (Tjong Kim Sang & De Meulder, CoNLL 2003)](https://aclanthology.org/W03-0419/).

❓ Check out the [Dataset card of the CoNLL 2003 dataset](https://huggingface.co/datasets/conll2003) on HuggingFace models.

❓ If you are still unsure how the BIO scheme works, now is the time to do a brief web search and figure it out! Advanced: The BILOU variant is currently achieving state-of-the-art results. Read about it.

In [ ]:
# Load the dataset
train_data = load_dataset("conll2003", split="train")
val_data =  load_dataset("conll2003", split="validation")
test_data =  load_dataset("conll2003", split="test")

❓ Print out the number of instances in each datasplit.

In [ ]:
# Your code here

The NER tags in the CoNLL 2003 NER dataset are (with their class indices):

```
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
```

❓ Create a list `ner_tags` of the labels in the order indicated by their values in the dictionary above. Create two dictionaries `tag2idx` and `idx2tag` that map from tag to class index and from class index to tag.

In [ ]:
# Your code here

❓ LOOK AT THE DATA. Print out a few examples and make sure you understand the data structure.

Example Output (training set instance with index 189):
```
Iraqi           B-MISC
President       O    
Saddam          B-PER
Hussein         I-PER
has             O    
told            O    
visiting        O    
Russian         B-MISC
ultra-nationalist O    
Vladimir        B-PER
Zhirinovsky     I-PER
that            O    
Baghdad         B-LOC
wanted          O    
to              O    
maintain        O    
"               O    
friendship      O    
and             O    
cooperation     O    
"               O    
with            O    
Moscow          B-LOC
,               O    
official        O    
Iraqi           B-MISC
newspapers      O    
said            O    
on              O    
Thursday        O    
.               O    
```

In [ ]:
# Your code here


### Tokenization

For tokenization, we use the [AutoTokenizer](https://huggingface.co/docs/transformers/model_doc/auto) class of the HuggingFace transformers library. It conveniently instantiates an object of the correct class depending on the model configured in the path.
The parameter setting `add_special_tokens=True` means that the `[CLS]` and the `[SEP]` tokens are added to the input. Even if we are not performing sentence classification or sentence pair classification, we have to add these tokens because BERT saw them during pre-training, and now expects to see them, too.

To save some computational resources, we will work with [TinyBERT](https://arxiv.org/abs/1909.10351) using the [model](https://huggingface.co/prajjwal1/bert-tiny) provided by [Bhargava et al., 2021](https://aclanthology.org/2021.insights-1.18/). TinyBERT has been trained using distillation to mimick the behavior of BERT, but it is a much smaller and more efficient model.

If `tokenizer.is_fast` is `True`, we are using a [Fast tokenizer](https://huggingface.co/learn/nlp-course/chapter6/3) that provides some special features that are quite handy when working with pre-trained models.

* As input, a fast tokenizer accepts either a string (a single input instance), a list of strings (input texts), or a list of a list of tokens. The latter is particularly helpful if our data is already pretokenized. In the CoNLL 2003 dataset, this is the case. (Note that we will use a gold standard tokenization for the "real" tokens. In a real-world setting with an automatic tokenizer, performance might be less. However, as long as we note this in our experimental section as a caveat, doing this is accepted nowadays because tokenizers work pretty well for many domains and genres.)

* If given more than one instance, the fast tokenizer automatically process the inputs in a batch, being MUCH fast than when processing individual instances. (Hint (optional exercise): Modify the code below such that it only processes one sentence at a time and see how much slower it gets.)

* The tokenizer accepts a wide range of useful parameter settings. The `return_tensors='pt'` options means that we want to get back PyTorch tensors. We add special tokens and we also want to get attention masks. Finally, we tell the tokenizer that the input is already split into words such that it does not attempt to create a tokenization that would contradict the given tokenization, which would be problematic if we want to map the outputs to the gold standard labels. Can you figure out what the `truncation` and the `padding` options do?

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny") # Load the tokenizer that comes with the TinyBERT model.
print("Is fast encoder (should be True):", tokenizer.is_fast)

input_examples = [["I", "visited", "the", "Augsburger", "Puppenkiste", "."], \
                  ["The", "Rathaus", "is", "much", "more", "interesting", "in", "my", "opinion", "."]]
encodings = tokenizer(input_examples, return_tensors='pt', add_special_tokens=True, \
                      return_attention_mask=True, is_split_into_words=True, \
                      truncation=True, padding=True, max_length=32)

print(type(encodings))
print(encodings)

Inspect the data structures returned by the tokenizer. They are of type [`BatchEncoding`](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/tokenizer#transformers.BatchEncoding).
The maximum length of the sequence has been determined based on the maximum input length of the training set / maximum model input size / the value of max_length that we defined.

The BatchEncoding object can be viewed as a dictionary where each entry contains a tensor that contains the data for one input sequence per row, i.e., the `input_ids`, `attention_mask` etc. of each instance are split across these tensors. However, the same row in these tensors always corresponds to the same instance.

You can print them out as follows.

In [ ]:
num = 0 # Change to 1 and check the outputs.
print(encodings["input_ids"][num])
print(encodings["attention_mask"][num])
print(encodings.tokens(num)) # This prints the word piece tokens (input_ids --> convert_to_token_ids)

Now, let's assume that our gold standard labels are as follows:

```
I           O
visited     O
the         O
Augsburger  B-ORG
Puppenkiste I-ORG
.           O
```

At this point, we have a problem: The original labels above relate to the "real" tokens, but the word-piece tokens have split up some of them into several tokens. Recall that we have only labels for "real" tokens. There are two possible strategies to address this:

1. Use the first subword token of a "real" token to represent the embedding, only compare the predicted label for this token to the gold standard token and back-propagate the loss accordingly only from these tokens. This can be achieved by setting the labels of all non-used subword tokens to `-100`, a magic label index ignored by PyTorch loss functions (i.e., no loss is backpropagated from these outputs).

2. Duplicate the label of the "real" token for all its subword tokens.

Today, we will implement version 1. Luckily, the `BatchEncoding` object has a method called `word_ids` which returns a list showing to which "real" token a subword token corresponds. Each entry in this list corresponds to the subword token at the same position. The numbers in this list indicate the position of the corresponding "real" token in the original sequence.

❓ Make sure you understand the output of the code cell below.

In [ ]:
num = 0 # change to 1 to inspect data
print([(i, w) for i, w in enumerate(input_examples[num])])
print(encodings.tokens(num))
word_ids = encodings.word_ids(num)
print(word_ids)
print([(orig_word_id, subword_token) for orig_word_id, subword_token in zip(word_ids, encodings.tokens(num))])

### Perform Tokenization and Creating Label Tensors

❓ Write a function that tokenizes the text of each input example using the tokenizer in the configuration as above. Use a `max_length` of 64. The original datasets provide the label information (NER tags) in the field `"ner_tags"`. Indices in this list correspond to the pre-tokenized "real" tokens. Create list of labels for each instance that contains the integer values of the class indices for each subwork token. Follow the idea that all subword tokens that do not correspond to the first subword token of a "real" token should get a label of `-100`. Tokenize and create label lists for each of the data splits.

In [ ]:
# Your code here

### Custom PyTorch Dataset for NER

❓ Write a custom PyTorch dataset, e.g., called `NERDataset` that inherits from `Dataset` and implements the `__init__`, `__len__`, and `__getitem__` methods. (Hint: Check out the notebooks from the last session if you are unsure how to do this.)

In [ ]:
# Your code here

### Adapting BERT for NER

❓ Next, complete the code for the `BertNerModel` below. The `forward` method should use one linear layer that outputs one logit per valid label (referring to the `ner_tags`, -100 is not a valid label). The input to this linear layer is simply the `last_hidden_state` of the BERT model.

In [ ]:
hidden_size = 128
num_tags = len(ner_tags)

class BertNerModel(torch.nn.Module):

  def __init__(self):
    # Your code here
    pass

  def forward(self, inputs):
    # Your code here
    pass

### Evaluation

Now, before we actually start the learning, we should define how we measure performance.

❓ Could we simply use token-wise accuracy? Why is this not meaningful?

❓ Work through [Evaluate sequence models in python](https://www.depends-on-the-definition.com/evaluate-sequence-models/) by Torbias Sterbak.
We can load the `seqeval` metric in PyTorch as follows.

```
import evaluate
metric = evaluate.load("seqeval")
```

The `metric` object can be called as follows:

`metric.compute(predictions=predictions, references=true_labels)`

Note that you should ignore any tokens that have a label of `-100`, i.e., from the model's output, you first need to create the `predictions` and `true_labels` lists. For example, assume the model output and the list of true labels (that you created above) look like this:

```
gold_labels =      [-100, 0, 0, 2, 3, -100, 3, 0, 4, -100, 0, 7, -100]
predicted_labels = [9,    0, 0, 1, 3,    3, 3, 0, 5,    5, 0, 7,    0]
```

You need to filter this list of labels such that all the `-100` values are removed:
```
gold_labels =      [0, 0, 2, 3, 3, 0, 4, 0, 7]
predicted_labels = [0, 0, 1, 3, 3, 0, 5, 0, 7]
```

❓ Implement a function (or two if you prefer) that (given a model):

1. Collect the gold standard labels and the predicted labels for all instances of a given dataset (hint: assume that the input is a DataLoader).

2. Filter the lists for valid tokens and labels as described above.

3. Load the sequeval metric and use `metric.compute` to compute class-wise and overall evaluation scores.

IMPORTANT HINT: Read through the next exercise first (implementation of the training loop) or adapt your functions later to make it fit to the model outputs.

In [ ]:
# Your code here

### Training

At the beginning of your training cell, include the following code for setting the random seeds. (It is important to execute any time you start the training to ensure the random seeds are reset at this point in time.)

❓ Implement the training loop (adapt if from last week's notebook). Use AdamW as optimizer (`optimizer = AdamW(model.parameters(), lr=learning_rate, betas=betas, eps=epsilon)`).
Suggested parameters:

```
num_epochs = 16
batch_size = 64
learning_rate = 3e-5
betas=(0.9,0.999)
epsilon=1e-08
```

Make sure to shuffle the training data in your training DataLoader.
The training takes about X minutes on a GPU T4 in Google Colab with my implementation, and the results for the test set are as follows (yours might differ a bit due to some randomness in the loss function works).

```
LOC
	      precision 0.8064516129032258
	         recall 0.8750754375377188
	             f1 0.8393632416787264
	         number 1657
MISC
	      precision 0.6221928665785997
	         recall 0.6709401709401709
	             f1 0.6456477039067854
	         number 702
ORG
	      precision 0.6782922429344558
	         recall 0.6807483403741702
	             f1 0.6795180722891565
	         number 1657
PER
	      precision 0.8291062801932367
	         recall 0.872299872935197
	             f1 0.8501547987616099
	         number 1574
			 
	overall_precision 0.7528089887640449
	 overall_recall 0.7910554561717352
	     overall_f1 0.7714584787159804
	overall_accuracy 0.9562975321214222
```

As loss function, use `torch.nn.CrossEntropyLoss()`, which INCLUDES the softmax already. When you determine the predictions from the output in your evaluation functions above, you do this simply by finding the argmax over the logit values.

Here is a snippet of PyTorch code that you may find useful:

```
    y_pred = model(X)  # Have our model with current weights make a prediction
    # outputs should be of shape [batch, sequence, logits] (where sequence values indicate the token indices within one sequence)
    y_pred = torch.permute(y_pred, (0, 2, 1))  # swap the sequence and the logit dimensions
    loss = loss_fn(y_pred, y)  # ... sucht that the loss function can take care of the rest for us!
```


In [ ]:
# Always fun with the random seeds ...
# We need to set them such that our results will be replicable.
# (Hint: for an experiment later, you can change the random seed here and check what happens.
# But for now, let's keep the answer to all questions of the universe, 42.)
seed=42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
if torch.cuda.is_available():
  # This is needed on Colab as we are working in a distributed environment
  # If you are working in a different GPU environment, you can probably omit this line if it results in errors.
  os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"


#####################################
# Instantiate the model             #
#####################################

# Your code here

#####################################
# Training / Fine-tuning the model  #
#####################################

# Your code here

In [ ]:
# Evaluate on the test set
# Your code here

❗ Upon completion, upload your code (this notebook) to your GitLab repository.